# Turkey exchange rate analysis
Data is from World Bank, see what we can learn from the data

Load needed packages

In [20]:
library(xlsx)
library(reshape2)
library(dplyr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [12]:
ex_rate <- read.xlsx("kaggle/input/global-economic-monitor/Exchange rate, new LCU per USD extended backward, period average.xlsx",1)

In [13]:
names(ex_rate)[1] <- 'year'
head(ex_rate)

year,Afghanistan,Angola,Albania,United.Arab.Emirates,Argentina,Armenia,Developing.Asia,Antigua.and.Barbuda,Australia,...,St..Vincent.and.the.Grenadines,Venezuela..RB,Vietnam,Vanuatu,World..WBG.members.,Samoa,Yemen..Rep.,South.Africa,Zambia,Zimbabwe
,,,,,,,,,,...,,,,,,,,,,
1990,NA,NA,NA,3.674964,NA,NA,242.0292,NA,1.279588,...,NA,NA,NA,NA,90.30874,NA,NA,2.5874,NA,NA
1991,NA,NA,NA,3.675188,NA,NA,282.4463,NA,1.283241,...,NA,NA,NA,NA,89.96299,NA,NA,2.759186,NA,NA
1992,NA,NA,NA,3.673874,0.991757,NA,290.6905,NA,1.361321,...,NA,NA,NA,NA,91.50781,NA,NA,2.853479,NA,NA
1993,NA,4903.561,110.0993,3.675296,0.998979,NA,299.6222,2.69682,1.471305,...,NA,NA,10698.57,122.3616,92.14131,NA,16.48504,3.269305,NA,NA
1994,NA,122750.8,104.3122,3.674759,0.998998,NA,310.0288,2.699279,1.36714,...,NA,NA,10945.17,115.0205,261.9977,NA,30.28067,3.548696,0.675617,NA


In [23]:
tail(ex_rate)

,year,Afghanistan,Angola,Albania,United.Arab.Emirates,Argentina,Armenia,Developing.Asia,Antigua.and.Barbuda,Australia,...,St..Vincent.and.the.Grenadines,Venezuela..RB,Vietnam,Vanuatu,World..WBG.members.,Samoa,Yemen..Rep.,South.Africa,Zambia,Zimbabwe
26,2014,57.2475,98.41658,105.5438,3.673,8.120707,415.9198,1188.709,2.7,1.109586,...,2.7,6.292221,21198.86,97.18582,429.966,2.331331,215.0249,10.84821,6.159591,322.3492
27,2015,61.14346,120.0968,125.8852,3.672992,9.256635,477.9183,1315.018,2.7,1.331668,...,2.7,6.292,21920.18,106.681,481.8392,2.545019,214.9747,12.7675,8.632821,321.5616
28,2016,67.86609,165.2109,124.1759,3.672996,14.75874,480.4882,1315.485,2.696728,1.34498,...,2.7,9.264732,22364.71,109.5311,510.0345,2.543874,240.7871,14.7061,10.31242,262.4624
29,2017,68.0269,167.0638,119.0652,3.673,16.54435,482.7164,1325.914,2.683555,1.304616,...,2.7,10.04022,22713.72,107.5134,550.537,2.49569,250.2535,13.3143,9.540511,167.4566
30,2018,71.74423,253.3759,108.0103,3.673,28.14307,482.9879,1395.645,2.697907,1.338981,...,2.7,77629.49,23014.95,108.0246,1099.669,2.539147,250.2623,13.24647,10.47366,72.51145
31,2019,NA,317.1089,109.9761,3.673,41.0368,NA,1394.253,2.704695,1.409718,...,NA,NA,23219.03,111.8691,1109.034,2.583549,250.3062,14.12217,12.20482,9.177661


In [16]:
ex_rate_melt <- melt(ex_rate,id=c("year"))

Warning message:
"attributes are not identical across measure variables; they will be dropped"

In [17]:
head(ex_rate_melt)

year,variable,value
,Afghanistan,
1990,Afghanistan,NA
1991,Afghanistan,NA
1992,Afghanistan,NA
1993,Afghanistan,NA
1994,Afghanistan,NA


From below we can see all the countries invovled in this data set, after checking I know the exchange rate of turkey is in the data set as well.

In [21]:
distinct(ex_rate_melt,variable)

variable
Afghanistan
Angola
Albania
United.Arab.Emirates
Argentina
Armenia
Developing.Asia
Antigua.and.Barbuda
Australia
Austria


By using below script we can check if there is missing data

In [22]:
apply(is.na(ex_rate),2,sum)

year                          Afghanistan 
                                   0                                    0 
                              Angola                              Albania 
                                   0                                    0 
                United.Arab.Emirates                            Argentina 
                                   0                                    0 
                             Armenia                      Developing.Asia 
                                   0                                    0 
                 Antigua.and.Barbuda                            Australia 
                                   0                                    0 
                             Austria                           Azerbaijan 
                                   0                                    0 
                             Burundi                              Belgium 
                                   0                                    0 
                               Benin                         Burkina.Faso 
                                   0                                    0 
                          Bangladesh                             Bulgaria 
                                   0                                    0 
                             Bahrain                         Bahamas..The 
                                   0                                    0 
              Bosnia.and.Herzegovina                              Belarus 
                                   0                                    0 
                              Belize                              Bolivia 
                                   0                                    0 
                              Brazil                             Barbados 
                                   0                                    0 
                   Brunei.Darussalam                               Bhutan 
                                   0                                    0 
                            Botswana             Central.African.Republic 
                                   0                                    0 
                              Canada                          Switzerland 
                                   0                                    0 
                               Chile                                China 
                                   0                                    0 
                       Cote.d.Ivoire                             Cameroon 
                                   0                                    0 
                    Congo..Dem..Rep.                          Congo..Rep. 
                                   0                                    0 
                            Colombia                              Comoros 
                                   0                                    0 
                          Cabo.Verde                           Costa.Rica 
                                   0                                    0 
                              Cyprus                       Czech.Republic 
                                   0                                    0 
                             Germany                 Developing.Countries 
                                   0                                    0 
                            Djibouti                             Dominica 
                                   0                                    0 
                             Denmark                   Dominican.Republic 
                                   0                                    0 
                             Algeria       East.Asia...Pacific.developing 
                                   0                                    0 
    Europe...Central.Asia.developing                              Ecuador 
                                   0                     

Looks like there is no NA, while it is obviously there are some NAs, let me use the melted data, and convert NA to real NA

In [33]:
ex_rate_melt_NA <- ex_rate_melt %>% 
  mutate(value = as.character(value)) %>% 
  mutate(value = case_when(
    value == "NA" ~ NA_character_, 
    TRUE ~ value
  ))

In [34]:
head(ex_rate_melt_NA)

year,variable,value
,Afghanistan,
1990,Afghanistan,NA
1991,Afghanistan,NA
1992,Afghanistan,NA
1993,Afghanistan,NA
1994,Afghanistan,NA


In [37]:
apply(is.na(ex_rate_melt),2,sum)

year variable    value 
       0        0        0

In [36]:
apply(is.na(ex_rate_melt_NA),2,sum)

year variable    value 
       0        0      854

It seems I "successfully" got lots of NAs now... Let me fix it tomorrow